## Лабораторна робота №2: "Імплементація алгоритмів стиснення"

Склад команди та розподіл виконаних завдань:

-
-

Для кожного з алгоритмів поданих нижче
- опишіть як працює алгорит
- напишіть класи з методами encode та decode
- перевірте правильність кодування та декодування
- дослідіть час виконання коду в залежності від розмірів вхідних даних
- оцініть ступінь стиснення(у відсотка) в залежності від розмірів
- напишіть висновок про ефективність різних алгоритмів та умови за яких той чи інший алгоритм дають кращий результат

# Алгоритм Гаффмана

В цьому алгоритмі доцільно імплементувати клас node та додаткові функції в Huffman для побудови дерева кодування

In [ ]:
class Huffman:
    @staticmethod
    def sort_list(lst):
        items = [(v, k) for k, v in lst.items()]
        items.sort(reverse=True)
        return {k: v for v, k in items}
    @staticmethod
    def create_table(data):
        list_of_nodes = []
        for i in set(data):
            list_of_nodes.append(Node(i, data.count(i)))
        list_of_nodes = sorted(list_of_nodes, key=lambda x: x.weight, reverse = True)
        temp1 = list_of_nodes.copy()
        list_of_trees = []
        list_of_trees.append(temp1.copy())
        while len(temp1) > 2:
            temp = []
            for i in range(len(temp1)-2):
                temp.append(temp1[i])
            rightnode = temp1[-2]
            leftnode = temp1[-1]
            temp.append(Node(leftnode.name + rightnode.name, leftnode.weight + rightnode.weight, None, leftnode, rightnode))
            temp1 = temp
            temp1 = sorted(temp1,  key=lambda x: x.weight, reverse = True)
            list_of_trees.append(sorted(temp1,  key=lambda x: x.weight, reverse = True))
        temp1[0].code = '1'
        temp1[1].code = '0'
        for i in list_of_trees[-2::-1]:
            # print(i)
            for nodee in i:
                if nodee.code == '':
                    if nodee.parent.left == nodee:
                        nodee.code += nodee.parent.code + '0'
                    else:
                        nodee.code += nodee.parent.code + '1'
                else:
                    continue
        leftnode = temp1[1]
        rightnode = temp1[0]
        return list_of_trees[0], Node(leftnode.name + rightnode.name, leftnode.weight + rightnode.weight, None, leftnode, rightnode)
    @staticmethod
    def encode(data):
        temp, root_node = Huffman.create_table(data)
        lst = dict()    
        table = dict()
        result = []
        lst = dict()
        for x in temp:
            lst[x.name] = x.weight
        items = [(v, k) for k, v in lst.items()]
        items.sort(reverse=True)
        list_of_chars = [k for v, k in items]
        for x in temp:
            table[x.name] = x.code
        # print(table)
        for char in data:
            result.append(table[char])
        return ''.join(result), root_node
    @staticmethod
    def decode(data, root_node):
        result = ''

        index = 0
        temp = ''
        node = root_node
        while index < len(data):
            node = root_node
            for i in data[index:]:
                if node.left != None:
                    if i == '0':
                        temp += i
                        node = node.left
                        index += 1
                    else:
                        temp += i
                        node = node.right
                        index += 1
                else:
                    result += node.name
                    # index += len(node.code)
                    break
            if index == len(data):
                result += node.name
                break
            print(index, len(data))
        return result

In [ ]:
#Test Huffman
data = 'I love my mother very much!'
encoded = Huffman.encode(data)
decoded = Huffman.decode(encoded)
print(decoded == data)

In [ ]:
Процес кодування:

Аналіз частот:
Алгоритм підраховує частоту появи кожного символу вхідного тексту.
Побудова дерева Гаффмана:
Створіть список вузлів, де кожен вузол містить символ та його частоту.
Поки список не буде порожнім:
Виберіть два вузли з найменшими частотами.
Створіть нове дерево, де ці два вузли стануть дочірніми, а їх частоти сумуються.
Додайте нове дерево до списку.
Кореневий вузол отриманого дерева буде деревом Гаффмана.
Присвоєння кодів:
Символам присвоюються коди шляхом обходу дерева Гаффмана.
Кожен шлях від кореня до листа стає кодом відповідного символу.
0 використовується для позначення лівого дочірнього вузла, а 1 – для правого.
Кодування тексту:
Кожен символ вхідного тексту замінюється його кодом.
Процес декодування:

Побудова дерева Гаффмана:
Дерево Гаффмана відновлюється з коду, який використовується для стиснення.
Декодування тексту:
Код читається посимвольно.
Починаючи з кореня дерева Гаффмана, алгоритм переміщається по дереву:
0 – перехід до лівого дочірнього вузла.
1 – перехід до правого дочірнього вузла.
Коли досягається листок, символ, що відповідає цьому листу, додається до декодованого тексту.
Процес повторюється, поки не буде декодовано весь текст.
Переваги алгоритму Гаффмана:

Простий та ефективний алгоритм стиснення.
Добре підходить для стиснення текстових даних.
Легко реалізувати.
Недоліки алгоритму Гаффмана:

Не завжди дає оптимальне стиснення.
Ефективність залежить від частоти символів.
Приклади:

Алгоритм Гаффмана використовується в ZIP-архівах.
Також використовується в JPEG-зображеннях.

# Алгоритм LZW

In [ ]:
class LZW:
    def encode(self, data):
        dictionary = {chr(i):i for i in range(256)}
        next_val = 256
        temporary_for_check = ''
        compressed_data = []
        index = 0
        while index < len(data)-1:
            unit = data[index]
            if unit in dictionary.keys():
                temporary_for_check = unit
                for char_temp in data[index+1:]:
                    temporary_for_check += char_temp
                    if temporary_for_check in dictionary.keys():
                        continue
                    else:
                        index += len(temporary_for_check)-1
                        dictionary[temporary_for_check] = next_val
                        compressed_data.append(dictionary[temporary_for_check[:-1]])
                        next_val += 1
                        break
            if index == len(data) - 1:
                compressed_data.append(dictionary[temporary_for_check[-1]])
            temporary_for_check = ''
        return compressed_data
    def decompress(self, compressed_data):
        dictionary = {i:chr(i) for i in range(256)}
        next_value = 256
        previous_entry = dictionary[compressed_data[0]]
        decompressed_data = ''
        decompressed_data += previous_entry
        for i in compressed_data[1:]:
            if i in dictionary.keys():
                decompressed_data += dictionary[i]
                previous_entry += dictionary[i][0]
                if previous_entry not in dictionary.values():
                    dictionary[next_value] = previous_entry
                    next_value += 1
                previous_entry = dictionary[i]
            elif i == next_value:
                previous_entry += previous_entry[0]
                dictionary[next_value] = previous_entry
                decompressed_data += dictionary[next_value]

                previous_entry = dictionary[next_value]
                next_value += 1
        return decompressed_data


In [ ]:
lzw = LZW()
data = 'God bless America'
encoded = lzw.encode(data)
decoded = lzw.decode(encoded)
print(decoded == data)

In [ ]:
Ось основні кроки роботи алгоритму LZW:

1. Ініціалізація:
- Словник містить усі можливі окремі символи як записи з унікальними кодами (зазвичай починаючи з 0).
- Вхідний текст розглядається посимвольно.

2. Пошук найдовшої відповідності:
- Алгоритм сканує вхідний текст, починаючи з поточного символу, і шукає найдовшу послідовність символів, яка вже існує в словнику.

3. Кодування та оновлення словника:
- Якщо знайдено відповідність:
- Вихідний код призначається коду відповідної послідовності в словнику.
- До словника додається нова послідовність, яка на один символ довша за знайдену відповідність, разом із новим унікальним кодом.
- Якщо відповідність не знайдено:
- Вихідний код призначається коду одиночного символу з вхідного тексту.
- До словника додається нова послідовність із двох символів (поточний символ і наступний) разом із новим унікальним кодом.

4. Повторення:
- Алгоритм переходить до наступного символу вхідного тексту та повторює кроки 2-3, доки весь текст не буде оброблено.

Декодування:

Декодування LZW є дзеркальним відображенням процесу кодування. Отримуючи код, декодер шукає відповідну послідовність у словнику та виводить її символи. Потім до словника додається нова, довша послідовність, і процес повторюється для наступного коду.

Переваги LZW:

Простий та ефективний для текстових даних з повторюваними фрагментами.
Не потребує попереднього аналізу статистики символів.
Використовується в різних форматах, таких як GIF-зображення та формат PDF.
Недоліки LZW:

Може бути менш ефективним для даних без повторень.
Словник може динамічно зростати під час кодування, що може призвести до складнощів у декодуванні, якщо початковий словник не узгоджено.

# Алгоритм LZ77

Потрібно заміряти розміри саме тексту, проте для роботи доцільно використовувати список тюплів, тому для зручності варто імплементувати додаткові алгоритми _text2list та _list2text

In [ ]:
class LZ77:
    def __init__(self, buffer_size: int):
        self.buff_size = buffer_size

    def encode(self, text: str) -> str:
        compressed_data = []
        bufsize = self.buff_size
        pos = 0
        max_length = len(data)
        while pos < len(data):
            dist_match = 0
            len_match = 0
            for i in range(1, min(max_length, len(data) - pos) + 1):
                substring = data[pos:pos + i]
                pre_occur = data[max(0, pos - bufsize): pos].rfind(substring)
                if pos - pre_occur <= bufsize and pre_occur != -1 :
                    len_match = i
                    dist_match = pos - pre_occur
                else:
                    break
            if len_match > 0:
                if pos + len_match < len(data):
                    next_char = data[pos + len_match] 
                compressed_data.append((dist_match, len_match, next_char))
                pos += len_match + 1
            else:
                compressed_data.append((0, 0, data[pos]))
                pos += 1
        return compressed_data

    def decode(self, code: str) -> str:
        decompressed_data = []
        for item in compressed_data:
            dist_match, len_match, next_char = item
            if len_match > 0:
                start_pos = len(decompressed_data) - dist_match
                for i in range(len_match):
                    decompressed_data.append(decompressed_data[start_pos + i])
            decompressed_data.append(next_char)
        return ''.join(decompressed_data)

In [ ]:
lz77 = LZ77(6)
data = 'I want my money back'
encoded = lz77.encode(data)
decoded = lz77.decode(encoded)
print(decoded == data)

Алгоритм LZ77, розроблений Абраамом Лемпелем та Яковом Зівом у 1977 році, є алгоритмом стиснення даних без втрат. Він працює шляхом заміни повторюваних фрагментів даних посиланнями на їх попередні появи, що дозволяє зменшити загальний розмір даних.

Основні компоненти алгоритму LZ77:

Пошукове вікно (search window): Це вікно фіксованого розміру, яке зберігає вже оброблені дані.
Поточний символ (lookahead buffer): Символ, який зараз обробляється.
Словник (dictionary): (Неявний) словник, що містить усі можливі вже знайдені фрагменти даних.
Процес кодування LZ77:

Ініціалізація:

Пошукове вікно та поточний символ отримують початкові значення.
Словник порожній.
Пошук найдовшої відповідності:

Алгоритм шукає найдовший фрагмент у пошуковому вікні, який збігається з поточним символом і наступними символами.
Кодування:

Якщо знайдено відповідність:
Вихідний код складається з трьох елементів:
Відстань (offset): Кількість символів назад у пошуковому вікні, де починається відповідність.
Довжина (length): Кількість символів у відповідності.
Символ (next character): Символ, який йде після відповідності.
Відповідність видаляється з пошукового вікна.
Поточний символ оновлюється наступним символом після відповідності.
Якщо відповідність не знайдено:
Вихідний код містить лише символ (next character).
Поточний символ переміщується на один символ вперед.
Повторення:

Кроки 2-3 повторюються доти, доки всі дані не будуть оброблені.
Декодування LZ77:

Декодування є протилежним процесом кодування. Отримуючи код, декодер:

Використовує відстань та довжину, щоб знайти відповідний фрагмент у пошуковому вікні.
Додає знайдений фрагмент до вихідних даних.
Додає символ з коду до вихідних даних.
Оновлює пошукове вікно новими даними.
Переваги LZ77:

Простий та ефективний для даних з повторюваними фрагментами.
Не потребує попереднього аналізу статистики даних.
Використовується як основа для багатьох інших алгоритмів стиснення, таких як LZMA та DEFLATE (використовується в ZIP-файлах).
Недоліки LZ77:

Може бути менш ефективним для даних без повторень.
Розмір пошукового вікна впливає на ефективність та швидкість алгоритму.

# Алгоритм Deflate

In [ ]:
class Deflate:
    def __init__(self, buffer_size=6: int):
        self.buffer_size = buffer_size
    def encode(self, data):
        lz77 = LZ77(self.buffer_size)
        temp, node = Huffman.encode(data)
        return lz77.compress(temp), node
    def decode(self, data, node):
        lz77 = LZ77(self.buffer_size)
        return Huffman.decode(lz77.decompress(data), node)

In [ ]:
deflate = Deflate(6)
data = 'Long live Ukraine!'
encoded, node = deflate.encode(data)
decoded = deflate.decode(data)
print(decoded == data)

1. Алгоритм LZ77:

Знаходить повторювані фрагменти даних: DEFLATE використовує алгоритм LZ77 для пошуку повторюваних фрагментів даних у файлі. Ці фрагменти замінюються посиланнями на їх попередні появи, що дозволяє зменшити загальний розмір даних.
2. Кодування Huffman:

Присвоює коди різної довжини: Після того, як LZ77 знаходить повторювані фрагменти, DEFLATE використовує Huffman coding для присвоєння кодів різної довжини символам у даних. Символи, які зустрічаються частіше, отримують коротші коди, а рідкісні символи – довші коди. Це дозволяє додатково зменшити розмір даних.
Переваги DEFLATE:

Широке використання: DEFLATE є одним із найпоширеніших алгоритмів стиснення без втрат і використовується в багатьох форматах, таких як ZIP, GZIP, PNG, та PDF.
Ефективність: Він добре підходить для стиснення текстових та інших даних, які містять повторювані фрагменти.
Відносна простота: DEFLATE є відносно простим алгоритмом, що робить його ефективним для реалізації в програмному забезпеченні.
Недоліки DEFLATE:

Не завжди оптимальний: DEFLATE не завжди забезпечує найкращий рівень стиснення для всіх типів даних.
Може бути повільнішим за інші алгоритми: Існує багато інших алгоритмів стиснення, які можуть бути швидшими, ніж DEFLATE, але при цьому вони можуть пропонувати гірший рівень стиснення.